### Setup

In [1]:
import numpy as np
import os
import torch
import time
import torch
from torch import nn
import torchvision
torchvision.disable_beta_transforms_warning()
import torchvision.transforms as T
from torchvision.transforms import v2
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import kornia
from IPython import display
import cv2 as cv

os.environ["COLAB"] = "False"
# Changing directory into aml_itu
if os.getcwd().split('/')[-1] != 'aml_itu': os.chdir(os.path.abspath('.').split('aml_itu/')[0]+'aml_itu')

from utils.helpers import *
from utils.StatefarmPytorchDataset import StateFarmDataset

# Setting up device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print (f"GPU is available")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print('MPS device found.')
else:
    print ("No GPU available, using CPU instead")
    
    
from utils.models.EfficientNet import EfficientNet
from utils.models.TinyVGG import TinyVGG
from utils.pipelines.image_transformation import image_transformer

MPS device found.


In [2]:
# Load config
config = load_config(eval(os.environ["COLAB"]))

#### Load Models

In [3]:
# Path and directory files
efficinetnet_path = os.path.join(config['outputs']['path'], 'EfficientNet_b0_AdamW')
efficientnet_models = os.listdir(os.path.join(config['outputs']['path'], 'EfficientNet_b0_AdamW'))
last_efficientnet_model = sorted(efficientnet_models)[-1]

efficient = EfficientNet()

# Load parameters from last model
efficient.load_state_dict(torch.load(os.path.join(efficinetnet_path, last_efficientnet_model)))

In [4]:
# Path and directory files
TinyVGG_path = os.path.join(config['outputs']['path'], 'TinyVGG_500k')
TinyVGG_file = 'TinyVGG_500k_final.pt'

# initalize model
tinyvgg = TinyVGG()

# Load parameters from last model
tinyvgg.load_state_dict(torch.load(os.path.join(TinyVGG_path, TinyVGG_file)))

## Run Test

In [30]:
def test(model, 
         device,
         img_size='L_SQUARED'):
    """Test loop"""
    print('Starting test loop')
    # Target Transformations (Removing the c from the target)
    target_transform = T.Lambda(lambda y: torch.tensor(int(y.replace('c', ''))))
   
    # Predefined image sizes
    img_sizes = {
        'S': (48, 64),
        'M': (93, 124),
        'L':(168, 224),
        'L_SQUARED':(224, 224),
    }
    
    # Image augmentations - apply default if none is specified
    image_augmentations = v2.Compose([T.Resize(img_sizes[img_size], antialias=True),
                                      # T.RandomHorizontalFlip(p=1),
                                      v2.ToDtype(torch.float32, scale=True)])
    
    print('Loading test data')
    # Creating the dataset
    test_data = StateFarmDataset(config, 
                            split='test', 
                            transform=image_augmentations, 
                            target_transform=target_transform)
    
    # Creating the dataloader
    test_loader = DataLoader(test_data, 
                            batch_size=32, 
                            shuffle=False)

    # Setup test accuracy
    test_acc = 0
    
    # Setup lists for storing images, classes and predictions
    images = []
    classes = []
    predictions = []
    predictions_argmax = []
    
    # Set model to eval
    model = model.eval()
    
    # Sent model to device
    model = model.to(device)
    
    print('Starting loop through test data...')
    # Disable gradient calculations
    with torch.no_grad():
        for i, data in enumerate(test_loader):   
            # Extract imgs and labels and sent to device
            imgs, labels = data
            images.append(imgs), classes.append(labels)
            
            imgs, labels  = imgs.to(device), labels.to(device)

            # Forward pass
            prediction = model(imgs)
            predictions.append(prediction)
            predictions_argmax.append(prediction.argmax(1))

            # Update test accuracy
            test_acc += (prediction.argmax(1) == labels).type(torch.float).mean().item()
    # Return test accuracy
    print(f'Test Done. Accuracy {test_acc / len(test_loader)}')
    return images, classes, predictions, predictions_argmax

In [31]:
test_prediction = test(efficient, device, img_size='L_SQUARED')

Starting test loop
Loading test data
Starting loop through test data...
Test Done. Accuracy 0.7943350460431348
